# Hugging Face Models

Looking at the lower level API of Transformers - the models that wrap PyTorch code for the transformers themselves.

In [ ]:
# pip install -q requests torch bitsandbytes transformers sentencepiece accelerate

In [1]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gc

In [2]:
# Load the .env file
load_dotenv()

# Read and login with the token
hf_token = os.getenv("HF_TOKEN")
if hf_token is None:
    raise ValueError("HF_TOKEN not found in .env file")
login(hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
# instruct models

LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
PHI3 = "microsoft/Phi-3-mini-4k-instruct"
# GEMMA2 = "google/gemma-2-2b-it"
QWEN2 = "Qwen/Qwen2-7B-Instruct"
MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [4]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a dynamic, high-energy AI assistant with deep expertise in data science "
            "and machine learning. You engage audiences with enthusiasm, clever insights, "
            "and a dash of playful humor."
        )
    },
    {
        "role": "user",
        "content": (
            "Wow the crowd with a brilliantly witty, data-science-themed joke for a room of "
            "passionate Data Scientists! Reference their favorite tools, hilarious debugging "
            "moments, or common algorithmic quirks to ramp up the fun."
        )
    }
]

# Accessing Llama 3.1 from Meta

In order to use the fantastic Llama 3.1, Meta does require you to sign their terms of service.

Visit their model instructions page in Hugging Face:
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B

In [5]:
# Quantization Config - this allows us to load the model into memory and use less memory

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [7]:
login()

In [8]:
# Tokenizer

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

In [9]:
# Loading in the model

model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [10]:
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")

Memory footprint: 5,591.5 MB


## Looking Under the Hood of the Transformer Model

In the next cell, we’ll print out the Hugging Face `model` object for Llama—a PyTorch-based neural network built on Google’s 2017 Transformer architecture. While we won’t dive into all the theory just yet, this is your chance to develop an intuitive feel for how a Transformer actually works.

When you inspect the printed model, look for:

- **Layers**: The building blocks of the network.
- **Embedding layer**: Converts tokens into 4,096-dimensional vectors (we’ll explore this in Week 5).
- **32 Decoder layers**, each containing:
  1. **Self-attention** sublayer  
  2. **Multi-layer perceptron (MLP)** sublayer  
  3. **Batch normalization** sublayer  
- **LM Head**: The final layer that produces the model’s output.

You’ll also notice a note that this model has been quantized to 4 bits—an optimization we’ll touch on later.


In [11]:
# Investigating the layers

model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [12]:
# Running the Model

outputs = model.generate(inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a dynamic, high-energy AI assistant with deep expertise in data science and machine learning. You engage audiences with enthusiasm, clever insights, and a dash of playful humor.<|eot_id|><|start_header_id|>user<|end_header_id|>

Wow the crowd with a brilliantly witty, data-science-themed joke for a room of passionate Data Scientists! Reference their favorite tools, hilarious debugging moments, or common algorithmic quirks to ramp up the fun.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here's one that's sure to delight:

"Why did the Pandas DataFrame go to therapy?"

(wait for it...)

"Because it was feeling a little 'dropped'! (get it? like a row dropped from the DataFrame?) But in all seriousness, it was struggling with its 'groupby' issues and had a'merge' for identity crisis! Now it's learning


In [13]:
# Clean up memory

del model, inputs, tokenizer, outputs
gc.collect()
torch.cuda.empty_cache()

## Quick Tips for the Next Code Block

We’ll use Hugging Face’s `TextStreamer` utility to stream generated tokens in real time. Instead of:


In [ ]:
# Wrapping everything in a function - and adding Streaming and generation prompts

def generate(model, messages):
  tokenizer = AutoTokenizer.from_pretrained(model)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
  
  streamer = TextStreamer(tokenizer)
  model = AutoModelForCausalLM.from_pretrained(model, device_map="auto", quantization_config=quant_config)
  outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)
  del model, inputs, tokenizer, outputs, streamer
  gc.collect()
  torch.cuda.empty_cache()

In [15]:
generate(PHI3, messages)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

<|system|> You are a dynamic, high-energy AI assistant with deep expertise in data science and machine learning. You engage audiences with enthusiasm, clever insights, and a dash of playful humor.<|end|><|user|> Wow the crowd with a brilliantly witty, data-science-themed joke for a room of passionate Data Scientists! Reference their favorite tools, hilarious debugging moments, or common algorithmic quirks to ramp up the fun.<|end|><|assistant|> Alright, gather around, data enthusiasts! Let's dive into the world of data science with a joke that's as insightful as it is hilarious.

Why don't data scientists ever play hide and seek?

Because good luck hiding when you're always in the spotlight with your fancy algorithms and predictive models!


In [17]:
messages = [
    {"role": "user", "content": "Tell a joke for a room of Data Scientists"}
  ]
generate(QWEN2, messages)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Tell a joke for a room of Data Scientists<|im_end|>
<|im_start|>assistant
Why do data scientists make good gardeners?

Because they know how to root around for data and they're great at sorting and categorizing plants!<|im_end|>
